In [ ]:
def access_trash():
  # First, authorize access to the Google Drive API
  from google.colab import auth
  from googleapiclient.discovery import build
  from googleapiclient.errors import HttpError

  auth.authenticate_user()
  drive_service = build('drive', 'v3')

  # Define the ID of the Trash folder
  trash_folder_id = 'trash'

  # Define the query to retrieve the files in the Trash folder
  query = "trashed=true"

  try:
    # Call the Drive API to retrieve the files in the Trash folder
    files = drive_service.files().list(q=query, spaces='drive').execute().get('files', [])

    # Iterate over the files in the Trash folder and delete them
    for file in files:
      try:
        drive_service.files().delete(fileId=file['id']).execute()
        print(f"File {file['name']} deleted from Trash folder.")
      except HttpError as error:
        print(f"An error occurred: {error}")
  except HttpError as error:
    print(f"An error occurred: {error}")

In [ ]:
!pip install allensdk
import allensdk
print(allensdk.__version__)

!pip install umap-learn

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import allensdk
from sklearn.decomposition import PCA
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from mpl_toolkits import mplot3d
import matplotlib.gridspec as gridspec
import umap.umap_ as umap

from allensdk.brain_observatory.behavior.behavior_project_cache.\
    behavior_neuropixels_project_cache \
    import VisualBehaviorNeuropixelsProjectCache

output_dir = './'

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive

cache = VisualBehaviorNeuropixelsProjectCache.from_s3_cache(
            cache_dir=Path(output_dir))
ecephys_sessions_table = cache.get_ecephys_session_table()
#ecephys_sessions_table.head()
''

Mounted at /content/gdrive
/content/gdrive/My Drive


''

In [27]:
def data(ecephy_session_id):

  #each session has tow session id, one is ecephy_session_id, one is behavior_session_id
  session = cache.get_ecephys_session(ecephys_session_id=ecephy_session_id)

  units = session.get_units()
  channels = session.get_channels()
  #same units with more columns, so we can seprate them based on more features
  unit_channels = units.merge(channels, left_on='peak_channel_id', right_index=True)

  #first let's sort our units by depth
  unit_channels = unit_channels.sort_values('probe_vertical_position', ascending=False)

  #now we'll filter them
  good_unit_filter = ((unit_channels['snr']>1)&
                    (unit_channels['isi_violations']<1)&
                    (unit_channels['firing_rate']>0.1))

  good_units = unit_channels.loc[good_unit_filter]
  print(len(units)-len(good_units), 'was removed due to bad conditions, not sure is a good idea')
  spike_times = session.spike_times

  stimulus_presentations = session.stimulus_presentations
  change_times = stimulus_presentations[stimulus_presentations['active']&
                            stimulus_presentations['is_change']]['start_time'].values

  return unit_channels,spike_times,change_times


In [ ]:
#Convenience function to compute the PSTH(histograms of the times at which neurons fire)
def makePSTH(spikes, startTimes, windowDur, binSize=0.001):
    bins = np.arange(0,windowDur+binSize,binSize)
    counts = np.zeros(bins.size-1)
    for i,start in enumerate(startTimes):
        startInd = np.searchsorted(spikes, start)
        endInd = np.searchsorted(spikes, start+windowDur)
        counts = counts + np.histogram(spikes[startInd:endInd]-start, bins)[0]
    
    counts = counts/startTimes.size
    return counts/binSize, bins


In [ ]:
#area_of_interest = 'VISp'

def binned_spike(area_of_interest,unit_channels,spike_times,change_times):

  if area_of_interest not in unit_channels.value_counts('structure_acronym'):
    return 0
    
  area_change_responses = []
  #here instead of good units(units with high filter we used units channels which is all the units)
  area_units = unit_channels[unit_channels['structure_acronym']==area_of_interest]
  #print((area_units.index))

  time_before_change = 1
  duration = 2.5
  for iu, unit in area_units.iterrows():
      unit_spike_times = spike_times[iu]
      unit_change_response, bins = makePSTH(unit_spike_times, 
                                            change_times-time_before_change, 
                                            duration, binSize=0.01)
      area_change_responses.append(unit_change_response)
  area_change_responses = np.array(area_change_responses)

  return area_change_responses

In [ ]:
#make it a func to call it severl 
def pca_plot2d(com_n, data,path):

  pca = PCA(n_components=com_n)
  pca_spike = pca.fit_transform(data)
  col=np.arange(0,len(data)) #the time binned
  #colormap
  cmap = mpl.cm.Reds
  norm = mpl.colors.Normalize(vmin=0, vmax=250)

  # Create a 2D scatter plot 
  fig, ax = plt.subplots()
  sc = ax.scatter(pca_spike[:,0], pca_spike[:,1],c = cmap(norm(col)))
  # Connect the dots with a line
  points = np.array([pca_spike[:,0], pca_spike[:,1]]).T.reshape(-1, 1, 2)
  segments = np.concatenate([points[:-1], points[1:]], axis=1)
  lc = LineCollection(segments, cmap='Reds')
  lc.set_array(col)
  ax.add_collection(lc)

  dx = np.diff(pca_spike[:,0])
  dy = np.diff(pca_spike[:,1])
  cmap = mpl.cm.get_cmap('Reds') # choose a color map
  colors = cmap(np.linspace(0, 1, len(dx))) # generate colors for each arrow based on dx

  for i in range(len(dx)):
      ax.annotate('', xy=(pca_spike[:,0][i+1], pca_spike[:,1][i+1]), xytext=(pca_spike[:,0][i], pca_spike[:,1][i]),
                  arrowprops=dict(arrowstyle='->', color=colors[i]))

  plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap))
  plt.xlabel("PC1")
  plt.ylabel("PC2")
  plt.title("PCA_"+area_of_interest)
  plt.savefig(path+"/PCA_Plot2d")
  plt.close()

  return pca.explained_variance_ratio_

In [ ]:
#make it a func to call it for all regions
def pca_plot3d(com_n, data,name,path,f=0,ax=None):

  pca = PCA(n_components=com_n)
  pca_spike = pca.fit_transform(data)
  col=np.arange(0,len(data)) #the time binned
  #colormap
  cmap = mpl.cm.Reds
  norm = mpl.colors.Normalize(vmin=0, vmax=250)

  # Create a 3D axes object pc1,pc2,pc3
  if f==0:
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot( projection='3d')

  ax.scatter(pca_spike[:,0], pca_spike[:,1], pca_spike[:,2],c = mpl.cm.cool(norm(col)))


  points = np.array([pca_spike[:,0], pca_spike[:,1], pca_spike[:,2]]).T.reshape(-1, 1, 3)
  segments = np.concatenate([points[:-1], points[1:]], axis=1)
  lc = Line3DCollection(segments, cmap='cool')
  lc.set_array(col)
  ax.add_collection(lc)

  if f==0:
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    ax.set_title('3D PCA_' + name + '_vs Time')

  plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.cool),fraction=0.025, pad=0.02)
  if f==0:
    plt.savefig(path+"/PCA_Plot3d")

  #plt.show()
  return pca.explained_variance_ratio_


In [34]:
def umap_plot3d(com_n,n_neigh,min_dist,data,name,path,f=0,ax=None): 
  
  np.random.seed(123)
  umap_model = umap.UMAP(n_components=com_n, n_neighbors=n_neigh, min_dist=min_dist)
  embedding = umap_model.fit_transform(data)

  col=np.arange(0,len(data)) #the time binned
  cmap = mpl.cm.Reds
  norm = mpl.colors.Normalize(vmin=0, vmax=250)

  if f==0:
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot( projection='3d')
    
  ax.scatter( embedding[:,0], embedding[:,1],embedding[:,2],c = mpl.cm.Reds(norm(col)))
  if f==0:
    ax.set_xlabel('UMAP 1')
    ax.set_ylabel('UMAP 2')
    ax.set_zlabel('UMAP 3')
    ax.set_title('UMAP Embeddings_'+ name)

  plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Reds),fraction=0.025, pad=0.02)
  if f==0:
    plt.savefig(path+"/Umap_Plot3d")

  #for combined data
  '''
  if f==2:
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot( projection='3d')
    ax.scatter( embedding[:,0], embedding[:,1],embedding[:,2],c = mpl.cm.Reds(norm(col)))
    plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Reds),fraction=0.025, pad=0.02)
    plt.savefig(path+'Umap_All region')
  '''
    
  #plt.close()



In [ ]:
id=1044385384
#unit_channels,spike_times,change_times = data(session_id)
#path= "/content/gdrive/My Drive/Results_neuropixel/Familiar/Umap_all" + str(session_id)
com_n=5
#for umap
n_neigh=5
min_dist=0.05

path= '/content/gdrive/My Drive/Results_neuropixel/Familiar'
reg=[]
all_data=[]
path = os.path.join(path, str(id))
# Loop through all the files in the directory and read them one by one
for file_name in os.listdir(path):
  if file_name.endswith('.npy'):
    file_path = os.path.join(path, file_name)
    data = np.load(file_path)
    reg.append(str(os.path.splitext(file_name)[0]))
    all_data.append(data)

all_inone = [item for sublist in all_data for item in sublist]

path= '/content/gdrive/My Drive/Results_neuropixel/Familiar/Umap_combined_data'  
path = os.path.join(path, str(id))
if not os.path.exists(path):
  os.makedirs(path)

umap_plot3d(com_n,n_neigh,min_dist,all_inone,('Umap_All region' +str(id)),path)


'''
#not needing this func anymore
#need to change the description of function
def combined_data(com_n,n_neigh,min_dist,path,data):
def combined_data(com_n,n_neigh,min_dist,path,unit_channels,spike_times,change_times):
  reg = ['VISp', 'VISpm', 'VISal', 'VISrl', 'VISl', 'VISam']
  all=[]

  for i in range(len(reg)):
    area_change_responses=binned_spike(reg[i],unit_channels,spike_times,change_times)
    all.append(area_change_responses)
  
  all_inone = [item for sublist in all for item in sublist]
  #all= all.reshape()
  #pca_plot3d(com_n, all_inone,'PCA_All region',path)
  #f==2 for combined data
  umap_plot3d(com_n,n_neigh,min_dist,all_inone,'Umap_All region',path,2)
'''

In [ ]:
'''
#need to be run on;y once, is Done for now
#preprocess the data , seprate the data of each region and save it in google drive so it wont take too long each time for analysis
#first for family session then for novel session

session_ids= ecephys_sessions_table.loc[(ecephys_sessions_table["experience_level"]=='Novel') & (ecephys_sessions_table["session_number"]==2)].index

savepath= "/content/gdrive/My Drive/Results_neuropixel/Novel"
reg = ['VISp', 'VISpm', 'VISal', 'VISrl', 'VISl', 'VISam']

#it has some problem catching 1047977240 this session, before it was ok :|
for id in session_ids:
  path=os.path.join(savepath, str(id))
  if not os.path.exists(path):
    os.makedirs(path)
  #os.makedirs(path)
  unit_channels,spike_times,change_times= data(id)
  for region in reg:
    reg_path = os.path.join(path, region)
    #if not os.path.exists(reg_path):
      #os.makedirs(reg_path)
    #construct the data for each reg
    area_change_responses= binned_spike(region,unit_channels,spike_times,change_times)
    if type(area_change_responses)==int:
      continue
    np.save(reg_path, area_change_responses)

  os.remove("./visual-behavior-neuropixels-0.4.0/behavior_ecephys_sessions/"+str(id)+"/ecephys_session_"+str(id)+".nwb")
  access_trash()
'''




In [ ]:
#compatible plot3D function to be used with gridspace
com_n=5
#for umap
n_neigh=15
min_dist=0.1
#path= "/content/gdrive/My Drive/Results_neuropixel"
#unit_channels,spike_times,change_times= data(1048189115)
#area_change_responses= binned_spike(area_of_interest,unit_channels,spike_times,change_times)
#def plot3d_forgridspec(method_name,com_n,n_neigh,min_dist,path,unit_channels,spike_times,change_times):

def plot3d_forgridspec(method_name,com_n,n_neigh,min_dist,path,data,reg):

    #reg = ['VISp', 'VISpm', 'VISal', 'VISrl', 'VISl', 'VISam']
    var=[]
    delt=[]
    var_s=[]
    #all=[]
    fig = plt.figure(figsize=(40,30))
    gs = fig.add_gridspec(len(reg), 2, width_ratios=[1,8], wspace=0.01)
    fig.subplots_adjust(wspace=0.05, hspace=0.1, left=0.1, right=0.25, top=0.95, bottom=0.1)
    #pca_plot3d(com_n, data,name,path,f=0,ax=None)
    #umap_plot3d(n_com,n_neigh,min_dist,data,name,path,f=0,ax=None)


# Set the path to the directory containing the files


    for i in range(len(reg)):
        ax = fig.add_subplot(gs[i, 1], projection='3d')
        #area_change_responses=binned_spike(reg[i],unit_channels,spike_times,change_times)
        #all.append(area_change_responses)
        #if type(area_change_responses)==int:
          #delt.append(reg[i])
          #continue

        if(method_name=='pca'):
          var.append(pca_plot3d(com_n, data[i].T, reg[i],path,1,ax))
          ax.set_xlabel('PC1' )
          ax.set_ylabel('PC2' )
          ax.set_zlabel('Time')
          ax.set_title('PCA_'+ reg[i])

        if(method_name=='umap'):
          umap_plot3d(com_n,n_neigh,min_dist, data[i].T, reg[i],path,1, ax)
          ax.set_xlabel('Embedding 1' )
          ax.set_ylabel('Embedding 2' )
          ax.set_zlabel('Embedding 3')
          ax.set_title('UMAP Embeddings_'+ reg[i])
        #ax = None # set the ax object to None to avoid conflict
        
    #print(len(delt))
    #if len(delt)==1:
      #reg.remove(delt[0])
    #if len(delt)>1 :
      #reg.remove(delt)
    
    
    if(method_name=='pca'):
      ax = fig.add_subplot(gs[:, 0])
      var_s=(sum(np.array(var).T))
      ax.plot(np.flip(var_s),np.flip(np.array(reg)),c='orange',marker='o')
      ax.set_xlabel('Sum_3PC_Variance')
      plt.xlim([0.6, 1])
      ax.set_title('3D PCA_vs Time')
    
    if(method_name=='pca'):
      plt.savefig(path+"/PCA_Plot3d_All")

    if(method_name=='umap'):
      plt.savefig(path+"/Umap_Plot3d_All")


#plot3d_forgridspec('pca',com_n,n_neigh,min_dist,path,unit_channels,spike_times,change_times)
#plot3d_forgridspec('umap',com_n,n_neigh,min_dist,path,unit_channels,spike_times,change_times)
#pca_plot3d(3, area_change_responses.T,'VISp',path,0)

In [ ]:
#ecephys_sessions_table.loc[(ecephys_sessions_table["experience_level"]=='Novel') & (ecephys_sessions_table["session_number"]==2)].index
#plot3d_forgridspec(method_name,com_n,n_neigh,min_dist,path,unit_channels,spike_times,change_times)


session_ids = ecephys_sessions_table.loc[(ecephys_sessions_table["experience_level"]=='Familiar') & (ecephys_sessions_table["session_number"]==1)].index
#print(type(session_ids))
session_ids=list(session_ids)
session_ids.remove([1047977240,1093642839,1095138995])
#cannot find this session data too ---> 1047977240, 1093642839 and 1095138995

com_n=5
#for umap
n_neigh=15
min_dist=0.1


for id in session_ids:
  path= '/content/gdrive/My Drive/Results_neuropixel/Familiar'
  reg=[]
  all_data=[]
  path = os.path.join(path, str(id))
# Loop through all the files in the directory and read them one by one
  for file_name in os.listdir(path):
    if file_name.endswith('.npy'):
      file_path = os.path.join(path, file_name)
      data = np.load(file_path)
      reg.append(str(os.path.splitext(file_name)[0]))
      all_data.append(data)

  path= '/content/gdrive/My Drive/Results_neuropixel/Familiar/PCA'
  savepath = os.path.join(path, str(id))
  if not os.path.exists(savepath):
    os.makedirs(savepath)
  #pca_plot3d(com_n, data.T,str(file_name),path,f=0)
  plot3d_forgridspec('pca',com_n,n_neigh,min_dist,savepath,all_data,reg)


  path= '/content/gdrive/My Drive/Results_neuropixel/Familiar/UMAP'
  savepath = os.path.join(path, str(id))
  if not os.path.exists(savepath):
    os.makedirs(savepath)
  plot3d_forgridspec('umap',com_n,n_neigh,min_dist,savepath,all_data,reg)




In [ ]:
#the same code for Novel sessions

session_ids = ecephys_sessions_table.loc[(ecephys_sessions_table["experience_level"]=='Novel') & (ecephys_sessions_table["session_number"]==2)].index
session_ids=list(session_ids)

com_n=5
#for umap
n_neigh=15
min_dist=0.1


for id in session_ids:
  path= '/content/gdrive/My Drive/Results_neuropixel/Novel'
  reg=[]
  all_data=[]
  path = os.path.join(path, str(id))
# Loop through all the files in the directory and read them one by one
  for file_name in os.listdir(path):
    if file_name.endswith('.npy'):
      file_path = os.path.join(path, file_name)
      data = np.load(file_path)
      reg.append(str(os.path.splitext(file_name)[0]))
      all_data.append(data)

  path= '/content/gdrive/My Drive/Results_neuropixel/Novel/PCA'
  savepath = os.path.join(path, str(id))
  if not os.path.exists(savepath):
    os.makedirs(savepath)
  #pca_plot3d(com_n, data.T,str(file_name),path,f=0)
  plot3d_forgridspec('pca',com_n,n_neigh,min_dist,savepath,all_data,reg)


  path= '/content/gdrive/My Drive/Results_neuropixel/Novel/UMAP'
  savepath = os.path.join(path, str(id))
  if not os.path.exists(savepath):
    os.makedirs(savepath)
  plot3d_forgridspec('umap',com_n,n_neigh,min_dist,savepath,all_data,reg)

In [ ]:
#The code for when you want to directly download the data and preprocess yourself and then use it.
'''
exp_var=[]
com_n=5
#for umap
n_neigh=15
min_dist=0.1
area_of_interest = 'VISrl'
savepath= "/content/gdrive/My Drive/Results_neuropixel"



for id in session_ids:
  path=os.path.join(savepath, str(id))
  if not os.path.exists(path):
    os.makedirs(path)
  #os.makedirs(path)
  unit_channels,spike_times,change_times= data(id)
  area_change_responses= binned_spike(area_of_interest,unit_channels,spike_times,change_times)
  #if the area_of_interest is not in the data in does not compute for that specific session.
  if type(area_change_responses)==int:
    continu
  exp_var.append(pca_plot2d(com_n, area_change_responses.T,path))
  pca_plot3d(com_n, area_change_responses.T,area_of_interest,path)
  plot3d_forgridspec('pca',com_n,n_neigh,min_dist,path,unit_channels,spike_times,change_times)
  plot3d_forgridspec('umap',com_n,n_neigh,min_dist,path,unit_channels,spike_times,change_times)
  os.remove("./visual-behavior-neuropixels-0.4.0/behavior_ecephys_sessions/"+str(id)+"/ecephys_session_"+str(id)+".nwb")
  access_trash()

np.savetxt(savepath+"/explained_variance",exp_var,delimiter=",")
'''

In [ ]:
#find the sessions for same mouse in order to compare them through time.
#for furthur analysis in the future
'''
mouse_id=ecephys_sessions_table['mouse_id']
grouped = ecephys_sessions_table.groupby('mouse_id').groups

ecephy_mouse=[]
for i in mouse_id:
  ecephy_mouse.append(grouped[i])

ecephy_mouse=list(set(tuple(x) for x in ecephy_mouse))
print(ecephy_mouse)
'''